In [1]:
# Chargement des librairies utilisées "pip install pandas selenium bs4"
import pandas
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [2]:
# Importation des données
df = pandas.read_csv("StockEtablissement_utf8_1000__complété.csv")
df

,siren,nic,siret,statutDiffusionEtablissement,dateCreationEtablissement,trancheEffectifsEtablissement,anneeEffectifsEtablissement,activitePrincipaleRegistreMetiersEtablissement,dateDernierTraitementEtablissement,etablissementSiege,...,etatAdministratifEtablissement,enseigne1Etablissement,enseigne2Etablissement,enseigne3Etablissement,denominationUsuelleEtablissement,activitePrincipaleEtablissement,nomenclatureActivitePrincipaleEtablissement,caractereEmployeurEtablissement,latitude,longitude
0,325175,16,32517500016,O,2000-09-26,NaN,NaN,3212ZZ,2015-03-18T00:58:59,False,...,F,NaN,NaN,NaN,NaN,32.12Z,NAFRev2,N,49.220518,6.015168
1,325175,24,32517500024,O,2008-05-20,NaN,NaN,NaN,2011-12-12T09:40:04,False,...,F,TAHITI PERLES CREATIONS,NaN,NaN,NaN,47.89Z,NAFRev2,N,43.938190,4.882656
2,325175,32,32517500032,O,2009-05-27,NaN,NaN,NaN,2014-07-08T00:10:21,False,...,F,TAHITI PERLES CREATIONS,NaN,NaN,NaN,32.12Z,NAFRev2,N,43.932374,4.844274
3,325175,40,32517500040,O,2011-10-21,NaN,NaN,3212ZZ,2015-03-18T00:58:59,False,...,F,TAHITI PERLES CREATION,NaN,NaN,NaN,32.12Z,NAFRev2,N,43.280158,5.623075
4,325175,57,32517500057,O,2014-01-07,NaN,NaN,NaN,2018-07-10T14:17:15,False,...,F,TAHITI PERLES CREATION,NaN,NaN,NaN,32.12Z,NAFRev2,N,43.299866,5.397044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,5742028,29,574202800029,O,2002-01-01,NN,NaN,NaN,2013-12-11T08:50:11,False,...,F,NaN,NaN,NaN,NaN,01.11Z,NAFRev2,N,44.115767,6.010904
995,5742036,14,574203600014,O,NaN,NaN,NaN,NaN,NaN,True,...,F,NaN,NaN,NaN,NaN,69.22,NAP,N,44.179140,5.947317
996,5742093,15,574209300015,O,NaN,NaN,NaN,NaN,2019-08-25T12:02:35,True,...,F,NaN,NaN,NaN,NaN,62.43,NAP,N,44.197272,5.944793
997,5742101,16,574210100016,O,1900-01-01,NN,NaN,NaN,2019-11-14T14:00:14,True,...,F,NaN,NaN,NaN,NaN,NaN,NaN,N,44.427401,6.435844


# Première étape : configurer le navigateur
Pour rappel, vous devez placer `chromedriver` dans le même répertoire que votre notebook (et le bon en fonction de votre OS).

In [3]:
driver = webdriver.Chrome()

## Création de l’URL à récupérer
On créé l’URL de la page qu’on souhaite récupérer. Pour faire une recherche dans Google Maps, il est préférable de mettre le nom de l’établissement, ainsi que les coordonnées géographiques de celui-ci.

In [8]:
base_url = "https://www.google.com/maps/search/"
place_info = "IUT+paris+rives+de+seine"
comp_url = "/@48.8489968,2.3125954,12z"

url = base_url + place_info + comp_url
url

'https://www.google.com/maps/search/IUT+paris+rives+de+seine/@48.8489968,2.3125954,12z'

# Premier lancement du navigateur
Pour rappel, **NE FERMEZ PAS LA FENETRE** mais cliquer sur Tout autoriser

In [9]:
driver.get(url)

## Fonction pour création de l'URL
On reprend la fonction vue dans la séance 1, avec une petite amélioration pour mettre le numéro de l'adresse en entier.

In [10]:
def f(v, fin = False):
    r = ""
    if not(pandas.isna(v)):
        if isinstance(v, float):
            r += str(int(v))
        else:
            r += str(v)
        if not(fin):
            r += "+"
    return r

### Code de recherche

In [11]:
# Code de recherche
site = []
for i in range(df.shape[0]):
    place_info = f(df["enseigne1Etablissement"][i]) + \
                 f(df["numeroVoieEtablissement"][i]) + f(df["indiceRepetitionEtablissement"][i]) + \
                 f(df["typeVoieEtablissement"][i]) + f(df["libelleVoieEtablissement"][i]) + \
                 f(df["codePostalEtablissement"][i]) + f(df["libelleCommuneEtablissement"][i], True)
    comp_url = "/@" +  str(df["latitude"][i]) + "," + str(df["longitude"][i]) + ",12z"
    url = base_url + place_info + comp_url
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html)
    results = soup.select("a[aria-label*='Site Web']")
    href = [None]
    if results != []:
        href += [results[0]["href"]]
    site += href
    time.sleep(1)

KeyboardInterrupt: 

### Affichage du résultat et stockage dans les données

In [12]:
site
df["site"] = site
df.to_csv("StockEtablissement_utf8_1000__avec_site.csv")

ValueError: Length of values (385) does not match length of index (999)